In [1]:
import pandas as pd
import numpy as np
import os
from scipy.optimize import curve_fit, Bounds
import colorcet as cc
from tqdm import tqdm

import bokeh.io
import bokeh.plotting
import bokeh.models
import iqplot

from utils import *
import importlib

# Specify the module name as a string
module_name = "1_calculate_viscoelastic_responce"

# Use importlib to import the module
calculate_viscoelastic_responce = importlib.import_module(module_name)

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
df_parameters = pd.read_csv(calculate_viscoelastic_responce.filepath_results_corrected, encoding='utf-8')

print(df_parameters['COMMENTS'].unique())
# df_parameters = df_parameters[df_parameters['COMMENTS'].isna()]
df_parameters.head(10)

[nan 'CHOSEN' 'CHOSEN, stuck beads excluded' 'spinning disk'
 'beads close to boundary\xa0'
 'CHOSEN, bead gets stuck a bit so potentially reconsider' 'CHX'
 'unclear, moved to interphase but beads might not be reached yet by MTs'
 'close to memb/ 1 bead outside embryo, moves to M' 'drift'
 'inconsistent, close to memb?' 'might move to M'
 'clearly M, all beads at membrane or beyond, movements into furrow'
 'beginning of I' 'unclear, transitioning to M'
 'beads close to boundary. Cell goes from M to interphase'
 'CHOSEN, bead between asters excluded' 'moves to interphase'
 'close to memb/ 1 bead outside embryo\xa0'
 "asters don't reach beads at beginning"
 'bead that is closer to tip is close to spindle'
 'beads are in fluid region between asters' 'bead in FMA - M-phase'
 'close to membrane?' 'close to memb/ 1 bead outside embryo'
 'unclear sample'
 'CHOSEN, 1 bead gets stuck at membrane after 270 time steps'
 'close to membrane' 'moving to M'
 'some beads in fluid region between aster

,Unnamed: 0,FILENAME,TRACK_IDX,PULSE_NUMBER,MT_STATUS,VISCOEL_PARAMS_RISING,VISCOEL_PARAMS_RELAXING,COMMENTS,VISCOEL_PARAMS_RISING_k,VISCOEL_PARAMS_RISING_eta1,VISCOEL_PARAMS_RISING_eta2,VISCOEL_PARAMS_RELAXING_a,VISCOEL_PARAMS_RELAXING_tau
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,20240118_s02p02t02_1_5sON_15sOFF_1000mV_1_exte...,3.0,4.0,y,NaN,NaN,CHOSEN,NaN,NaN,NaN,0.033163,1.454067
2,1,20240118_s02p02t02_1_5sON_15sOFF_1000mV_1_exte...,3.0,5.0,y,NaN,NaN,CHOSEN,30.319802,26.396232,163.163319,0.029480,1.363772
3,2,20240118_s02p02t02_1_5sON_15sOFF_1000mV_1_exte...,3.0,6.0,y,NaN,NaN,CHOSEN,NaN,NaN,NaN,0.054114,1.570718
4,3,20240118_s02p02t02_1_5sON_15sOFF_1000mV_1_exte...,3.0,7.0,y,NaN,NaN,CHOSEN,NaN,NaN,NaN,0.047074,1.311130
5,4,20240118_s02p02t02_1_5sON_15sOFF_1000mV_1_exte...,3.0,8.0,y,NaN,NaN,CHOSEN,NaN,NaN,NaN,0.030631,1.609547
6,5,20240118_s02p02t02_1_5sON_15sOFF_1000mV_1_exte...,3.0,9.0,y,NaN,NaN,CHOSEN,26.589887,14.269239,159.735763,0.018906,1.599751
7,6,20240118_s02p02t02_1_5sON_15sOFF_1000mV_1_exte...,3.0,10.0,y,NaN,NaN,CHOSEN,NaN,NaN,NaN,0.023457,1.658830
8,0,20231212_s02p01t03_1_5sON_15sOFF_1000mV_1_SELE...,4.0,1.0,n,NaN,NaN,"CHOSEN, stuck beads excluded",15.995308,16.000833,85.688275,0.024349,1.949226
9,1,20231212_s02p01t03_1_5sON_15sOFF_1000mV_1_SELE...,4.0,2.0,n,NaN,NaN,"CHOSEN, stuck beads excluded",NaN,NaN,NaN,0.012289,1.859482


In [3]:
# plot data
p = iqplot.stripbox(df_parameters, q='VISCOEL_PARAMS_RISING_k', cats='MT_STATUS')

bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_parameters, q='VISCOEL_PARAMS_RISING_eta1', cats='MT_STATUS')

bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_parameters, q='VISCOEL_PARAMS_RISING_eta2', cats='MT_STATUS')

bokeh.io.show(p)



In [4]:
# plot data
p = iqplot.stripbox(df_parameters, q='VISCOEL_PARAMS_RELAXING_a', cats='MT_STATUS')

bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_parameters, q='VISCOEL_PARAMS_RELAXING_tau', cats='MT_STATUS')

bokeh.io.show(p)


In [5]:
filepath_preliminary_results = '/Volumes/cytokinesis-zebrafish-collab/magnetic_tweezers_time_prec/3_plots/preliminary_results'

key_word = 'CHOSEN'
# to do: crop the 270 case
df_parameters['COMMENTS'] = df_parameters['COMMENTS'].astype(str)
df_flitered_results = df_parameters[df_parameters['COMMENTS'].str.contains(key_word)]
df_flitered_results.head()

# plot data
p = iqplot.stripbox(df_flitered_results, q='VISCOEL_PARAMS_RISING_k', cats='MT_STATUS', spread='swarm')
bokeh.io.export_png(p, filename=f'{filepath_preliminary_results}/chosen_data_elasticity_k.png')
bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_flitered_results, q='VISCOEL_PARAMS_RISING_eta1', cats='MT_STATUS', spread='swarm')
bokeh.io.export_png(p, filename=f'{filepath_preliminary_results}/chosen_data_viscosity_eta1.png')
bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_flitered_results, q='VISCOEL_PARAMS_RISING_eta2', cats='MT_STATUS', spread='swarm')
bokeh.io.export_png(p, filename=f'{filepath_preliminary_results}/chosen_data_viscosity_eta2.png')
bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_flitered_results, q='VISCOEL_PARAMS_RELAXING_a', cats='MT_STATUS', spread='swarm')
bokeh.io.export_png(p, filename=f'{filepath_preliminary_results}/chosen_data_offset_parameter_a.png')
bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_flitered_results, q='VISCOEL_PARAMS_RELAXING_tau', cats='MT_STATUS', spread='swarm')
bokeh.io.export_png(p, filename=f'{filepath_preliminary_results}/chosen_data_relaxation_time_tau.png')
bokeh.io.show(p)

In [6]:
# plot only confocal data 
key_word = 'spinning'
# to do: crop the 270 case
df_parameters['COMMENTS'] = df_parameters['COMMENTS'].astype(str)
df_flitered_results = df_parameters[(df_parameters['COMMENTS'].str.contains(key_word))&(df_parameters['VISCOEL_PARAMS_RISING_eta2']<50000)]
df_flitered_results.head()

save_data_to_filepath = '/Volumes/cytokinesis-zebrafish-collab/magnetic_tweezers_time_prec/3_plots/preliminary_results/'
# plot data
p = iqplot.stripbox(df_flitered_results, q='VISCOEL_PARAMS_RELAXING_a', cats='MT_STATUS', spread='jitter')
bokeh.io.export_png(p, filename=f'{filepath_preliminary_results}/spinning_disc_VISCOEL_PARAMS_RELAXING_a.png')
bokeh.io.save(p, save_data_to_filepath+'spinning_disc_VISCOEL_PARAMS_RELAXING_a.html')
# bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_flitered_results, q='VISCOEL_PARAMS_RELAXING_tau', cats='MT_STATUS', spread='jitter')
bokeh.io.export_png(p, filename=f'{filepath_preliminary_results}/spinning_disc_VISCOEL_PARAMS_RELAXING_tau.png')
bokeh.io.save(p, save_data_to_filepath+'spinning_disc_VISCOEL_PARAMS_RELAXING_tau.html')
# bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_flitered_results, q='VISCOEL_PARAMS_RISING_k', cats='MT_STATUS', spread='jitter')
bokeh.io.export_png(p, filename=f'{filepath_preliminary_results}/spinning_disc_VISCOEL_PARAMS_RISING_k.png')
bokeh.io.save(p, save_data_to_filepath+'spinning_disc_VISCOEL_PARAMS_RISING_k.html')
# bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_flitered_results, q='VISCOEL_PARAMS_RISING_eta1', cats='MT_STATUS', spread='jitter')
bokeh.io.export_png(p, filename=f'{filepath_preliminary_results}/spinning_disc_VISCOEL_PARAMS_RISING_eta1.png')
bokeh.io.save(p, save_data_to_filepath+'spinning_disc_VISCOEL_PARAMS_RISING_eta1.html')
# bokeh.io.show(p)

# plot data
p = iqplot.stripbox(df_flitered_results, q='VISCOEL_PARAMS_RISING_eta2', cats='MT_STATUS', spread='jitter')
bokeh.io.export_png(p, filename=f'{filepath_preliminary_results}/spinning_disc_VISCOEL_PARAMS_RISING_eta2.png')
bokeh.io.save(p, save_data_to_filepath+'spinning_disc_VISCOEL_PARAMS_RISING_eta2.html')
# bokeh.io.show(p)

/var/folders/c1/95zwm3_14_j1wp412s49dvlr0000gp/T/ipykernel_1783/3407545687.py:12: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  bokeh.io.save(p, save_data_to_filepath+'spinning_disc_VISCOEL_PARAMS_RELAXING_a.html')
/var/folders/c1/95zwm3_14_j1wp412s49dvlr0000gp/T/ipykernel_1783/3407545687.py:12: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  bokeh.io.save(p, save_data_to_filepath+'spinning_disc_VISCOEL_PARAMS_RELAXING_a.html')
/var/folders/c1/95zwm3_14_j1wp412s49dvlr0000gp/T/ipykernel_1783/3407545687.py:18: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  bokeh.io.save(p, save_data_to_filepath+'spinning_disc_VISCOEL_PARAMS_RELAXING_tau.html')
/var/folders/c1/95zwm3_14_j1wp412s49dvlr0000gp/T/ipykernel_1783/3407545687.py:18: UserWarning: save() called b

'/Volumes/cytokinesis-zebrafish-collab/magnetic_tweezers_time_prec/3_plots/preliminary_results/spinning_disc_VISCOEL_PARAMS_RISING_eta2.html'